# Robotics, Vision & Control 3e: for Python
## Chapter 6: Localization and Mapping

Copyright (c) 2021- Peter Corke

In [ ]:
try:
    import google.colab
    print('Running on CoLab')
    COLAB = True
    #!pip install roboticstoolbox-python>=1.0.2
    !pip install git+https://github.com/petercorke/robotics-toolbox-python@future
    !pip install spatialmath-python>=1.1.5
    !pip install --no-deps rvc3python
except ModuleNotFoundError:
    COLAB = False

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr_or_assign"
from IPython.display import HTML

import matplotlib.pyplot as plt

# add RTB examples folder to the path
import sys, os.path
import RVC3 as rvc
sys.path.append(os.path.join(rvc.__path__[0], 'models'))

# ------ standard imports ------ #
import numpy as np
import math
from math import pi
np.set_printoptions(
    linewidth=120, formatter={
        'float': lambda x: f"{0:8.4g}" if abs(x) < 1e-10 else f"{x:8.4g}"})
np.random.seed(0)
from spatialmath import *
from spatialmath.base import *
from roboticstoolbox import *

Running on CoLab
  Cloning https://github.com/petercorke/robotics-toolbox-python (to revision future) to /tmp/pip-req-build-eqrwj6dn
  Running command git clone --filter=blob:none --quiet https://github.com/petercorke/robotics-toolbox-python /tmp/pip-req-build-eqrwj6dn
  Running command git checkout -b future --track origin/future
  Switched to a new branch 'future'
  Branch 'future' set up to track remote branch 'future' from 'origin'.
  Resolved https://github.com/petercorke/robotics-toolbox-python to commit 6188446a943f6c4d830b4d79b0a36b45cd3e7084
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached spatialgeometry-1.1.0-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.5 kB)
  Using cached pgraph_python-0.6.3-py3-none-any.whl.metadata (7.5 kB)
  Using cached swift_sim-1.1.0-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.man

There are some minor code changes compared to the book. These are to support
animations in the Jupyter environment. Animations can be made using `FuncAnimation` and
the `widget` backend, but with Colab these can be very unsmooth.  Instead we render the
animations to HTML5 and render them locally in the browser.

# 6.1 Dead Reckoning using Odometry


## 6.1.1 Modeling the Robot


In [ ]:
V = np.diag([0.02, np.deg2rad(0.5)]) ** 2;

In [ ]:
robot = Bicycle(covar=V, animation="car")

In [ ]:
odo = robot.step((1, 0.3))

In [ ]:
robot.q

In [ ]:
robot.f([0, 0, 0], odo)

In [ ]:
robot.control = RandomPath(workspace=10)

In [ ]:
# robot.run(T=10);

html = robot.run_animation(T=20, format="html");
HTML(html)

## 6.1.2 Estimating Pose


In [ ]:
robot.Fx([0, 0, 0], [0.5, 0.1])

In [ ]:
x_sdev = [0.05, 0.05, np.deg2rad(0.5)];
P0 = np.diag(x_sdev) ** 2;

In [ ]:
ekf = EKF(robot=(robot, V), P0=P0)

In [ ]:
# ekf.run(T=20);

html = ekf.run_animation(T=20, format="html");
HTML(html)

In [ ]:
robot.plot_xy(color="b")
ekf.plot_xy(color="r")

In [ ]:
P150 = ekf.get_P(150)

In [ ]:
np.sqrt(P150[0, 0])

In [ ]:
ekf.plot_xy(color="r")
ekf.plot_ellipse(filled=True, facecolor="g", alpha=0.3)

In [ ]:
t = ekf.get_t();
pn = ekf.get_Pnorm();
plt.plot(t, pn);

# 6.2 Localizing with a Landmark Map


In [ ]:
map = LandmarkMap(20, workspace=10)

In [ ]:
map.plot()

In [ ]:
W = np.diag([0.1, np.deg2rad(1)]) ** 2;

In [ ]:
sensor = RangeBearingSensor(robot=robot, map=map, covar=W,
           angle=[-pi/2, pi/2], range=4, animate=True)

In [ ]:
z, i = sensor.reading()
# z
# i
print(f"landmark {i} at {z}")

In [ ]:
map[15]

In [ ]:
map = LandmarkMap(20, workspace=10);
V = np.diag([0.02, np.deg2rad(0.5)]) ** 2
robot = Bicycle(covar=V, animation="car");
robot.control = RandomPath(workspace=map, seed=0)
W = np.diag([0.1, np.deg2rad(1)]) ** 2
sensor = RangeBearingSensor(robot=robot, map=map, covar=W,
           angle=[-pi/2, pi/2], range=4, seed=0, animate=True);
P0 = np.diag([0.05, 0.05, np.deg2rad(0.5)]) ** 2;
ekf = EKF(robot=(robot, V), P0=P0, map=map, sensor=(sensor, W));

In [ ]:
# ekf.run(T=20)
html = ekf.run_animation(T=20, format="html")
HTML(html)

In [ ]:
map.plot()
robot.plot_xy();
ekf.plot_xy();
ekf.plot_ellipse()

# 6.3 Creating a Landmark Map


In [ ]:
map = LandmarkMap(20, workspace=10, seed=0);
robot = Bicycle(covar=V, animation="car");
robot.control = RandomPath(workspace=map);
W = np.diag([0.1, np.deg2rad(1)]) ** 2
sensor = RangeBearingSensor(robot=robot, map=map, covar=W,
           range=4, angle=[-pi/2, pi/2], animate=True);
ekf = EKF(robot=(robot, None), sensor=(sensor, W));

In [ ]:
# ekf.run(T=100);

html = ekf.run_animation(T=100, format="html")
HTML(html)

In [ ]:
map.plot();
ekf.plot_map();
robot.plot_xy();

In [ ]:
ekf.landmark(10)

In [ ]:
ekf.x_est[24:26]

In [ ]:
ekf.P_est[24:26, 24:26]